# Predict community from style features (vs unigrams)

## Load, prepare data

In [1]:
import pandas as pd

# Load data
data = pd.read_pickle('/mnt/interns/myoder/textposts_captions_1m.pkl')
print(len(data))
print(data.columns)

1000000
Index(['post_tags_string', 'post_id', 'activity_time_epoch', 'tumblelog_id',
       'is_private', 'post_title', 'post_short_url', 'post_slug', 'post_type',
       'post_caption', 'post_format', 'post_note_count', 'post_tags',
       'post_content', 'reblogged_from_post_id', 'reblogged_from_metadata',
       'created_time_epoch', 'updated_time_epoch', 'is_submission', 'mentions',
       'source_title', 'source_url', 'post_classifier', 'blog_classifier',
       'activity_date', 'post_body', 'post_body_no_blognames', 'text_type',
       'style_features', 'community'],
      dtype='object')


In [3]:
row_features = data['style_features'].map(lambda x: x.keys())
features = set([f for feats in row_features for f in feats])
    
print(len(features))

1417


In [4]:
# Expand dataframe with columns for features (probably don't need if using DictVectorizer)
import numpy as np
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, Manager

# for f in tqdm(features):
#     data[f] = data['style_features'].map(lambda x: x.get(f, 0))

m = Manager()
feature_d = m.dict()

def expand_feature(feature):
     feature_d[feature] = data['style_features'].map(lambda x: x.get(feature, 0))

with Pool(10) as p:
    list(tqdm(p.imap(expand_feature, features), total=len(features)))

Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-10:
Process ForkPoolWorker-8:
Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/homes/myoder/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/homes/myoder/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/homes/myoder/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/homes/myoder/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/homes/myoder/anaconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/homes/myoder/anaconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._ta

KeyboardInterrupt: 

In [ ]:
for feature, col in tqdm(feature_d.items()):
    data[feature] = col

print(data.shape)
data.columns.tolist()

In [ ]:
%xdel feature_d

## Split data, format for sklearn

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer

train, test = train_test_split(data, test_size=0.1, random_state=9)

# Select columns, normalize
# feature_cols = train.columns[data.columns.tolist().index('community') + 1:]
# X_train = train.loc[:, 'style_features']
# X_test = test.loc[:, 'style_features']

# Style features vectorizer
print("Fitting vectorizer...")
style_vectorizer = DictVectorizer()
style_vectorizer.fit(train['style_features'])

print("Running vectorizer...")
X_train = style_vectorizer.transform(train['style_features'])
X_test = style_vectorizer.transform(test['style_features'])

y_train = train['community']
y_test = test['community']

print("Normalizing...")
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Fitting vectorizer...
Running vectorizer...
Normalizing...
(900000, 1402)
(900000,)
(100000, 1402)
(100000,)


## Classify communities

In [ ]:
from sklearn.linear_model import LogisticRegression

# clf = LogisticRegression(solver='sag', multi_class='multinomial', verbose=2)
clf = LogisticRegression(solver='sag', multi_class='ovr', n_jobs=20, verbose=2)
clf.fit(X_train, y_train)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.


In [9]:
clf.score(X_test, y_test) # mean accuracy

0.32939293878587755

In [11]:
1/len(communities)

0.125